In [24]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random

In [25]:
x = torch.ones([1,2,2])
print(x)

tensor([[[1., 1.],
         [1., 1.]]])


In [40]:
class Module_conv_bn(nn.Module):
    def __init__(self) -> None:
        super(Module_conv_bn, self).__init__()
        self.conv = nn.Conv2d(1, 3, kernel_size=2, padding=1, stride=1)
        self.batch = nn.BatchNorm1d(3)
    def forward(self, x):
        x = self.conv(x)
        x = self.batch(x)
        return x, self.conv.weight.data, self.batch.weight.data

In [41]:
mod = Module_conv_bn()
res, wt_conv, wt_batch = mod(x)
print(res)

tensor([[[ 1.1232, -0.0112, -1.3246],
         [ 1.7090, -0.2784, -2.0913],
         [ 1.6978, -0.2236, -1.6009]],

        [[-0.6196, -0.6815, -1.1291],
         [ 0.4927,  0.7041, -0.5734],
         [ 0.7531,  1.2036, -0.4111]],

        [[-0.1182,  1.2502,  1.5109],
         [-0.4776, -0.0585,  0.5734],
         [ 0.0231, -1.0939, -0.3481]]], grad_fn=<NativeBatchNormBackward0>)


In [42]:
print(wt_batch)

tensor([1., 1., 1.])


In [43]:
class Only_Conv(nn.Module):
    def __init__(self) -> None:
        super(Only_Conv, self).__init__()
        self.new_conv = nn.Conv2d(1, 3, kernel_size=2, padding=1, stride=1)
        # I need to update date for that conv
        self.new_conv.weight.data=(wt_conv)
    def forward(self, x):
        x = self.conv(x)
        return x

In [44]:
new_mod = Only_Conv()
res, wt_conv, wt_batch = mod(x)
print(res)

tensor([[[ 1.1232, -0.0112, -1.3246],
         [ 1.7090, -0.2784, -2.0913],
         [ 1.6978, -0.2236, -1.6009]],

        [[-0.6196, -0.6815, -1.1291],
         [ 0.4927,  0.7041, -0.5734],
         [ 0.7531,  1.2036, -0.4111]],

        [[-0.1182,  1.2502,  1.5109],
         [-0.4776, -0.0585,  0.5734],
         [ 0.0231, -1.0939, -0.3481]]], grad_fn=<NativeBatchNormBackward0>)
